# Analyzing VIIRS imagery in AWS

This notebook focuses on two nighttime lights analyses: zonal statistics and timelapse mapping

In [ ]:
import sys
import os
import rasterio
import imageio

import pandas as pd
import geopandas as gpd


sys.path.insert(0, "../src")

import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.ntlMisc as ntl
from GOSTrocks.misc import tPrint

%load_ext autoreload
%autoreload 2

### Zonal statistics

In [ ]:
# Get a list of the VIIRS images in S3. This example leverages the GOST teams S3 bucket
ntl_files = ntl.aws_search_ntl()
ntl_files

In [ ]:
# set path to viirs folder
viirs_folder = "."

# define output path for output maps
out_map_folder = "/outfolder"

In [ ]:
ntl.aws_search_ntl?

In [ ]:
# Run zonal statistics against the define admin (in_zones)
in_zones = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inD = gpd.read_file(in_zones)
inD = inD.loc[inD["ISO3"] == "URY"]
inD = inD.to_crs("epsg:4326")

for cur_tif in ntl_files:
    res = rMisc.zonalStats(inD, cur_tif, minVal=0.05, reProj=True)
    res = pd.DataFrame(res, columns=["SUM", "MIN", "MAX", "MEAN"])
    inD[cur_tif.split("/")[5]] = res["SUM"]
    tPrint(os.path.basename(cur_tif))

pd.DataFrame(inD.drop(["geometry"], axis=1)).to_csv(f"{in_zones[:-4]}_NTL.csv")

In [ ]:
# If you want to clip out the raster data for the below mapping, run this block

in_zones = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inD = gpd.read_file(in_zones)
inD = inD.loc[inD["ISO3"] == "URY"]
inD = inD.to_crs("epsg:4326")

for cur_tif in ntl_files:
    file = f'{cur_tif.split("/")[-1]}.tif'
    out_file = os.path.join(os.path.join(viirs_folder, "%s" % file))
    curR = rasterio.open(cur_tif)
    if not os.path.exists(out_file):
        rMisc.clipRaster(curR, inD, out_file)

In [ ]:
# Clip out DMSP datasets
dmsp_folder = "/home/public/Data/GLOBAL/NighttimeLights/DMSP/"
for dmsp_file in os.listdir(dmsp_folder):
    curR = rasterio.open(os.path.join(dmsp_folder, dmsp_file))
    out_file = os.path.join(os.path.join(viirs_folder, "%s" % file))

# Generate maps automatically

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
# import earthpy as et
# import earthpy.plot as ep

# Prettier plotting with seaborn
import seaborn as sns

sns.set(font_scale=1.5, style="whitegrid")

all_files = [os.path.join(viirs_folder, x) for x in os.listdir(viirs_folder)]
all_files.sort()
all_files[:5]

In [ ]:
def map_viirs(cur_file, out_file=""):
    """create map of viirs data

    INPUT
        cur_file [string] - path to input geotif
        [optional] out_file [string] - path to create output image
    """
    # extract the year from the file name
    year = cur_file.split("_")[-1][:4]

    # Open the VIIRS data and reclassify
    inR = rasterio.open(cur_file)
    inD = inR.read()
    ### TODO: play with class_bins to change the colour scale
    class_bins = [-10, 0.5, 1, 2, 3, 5, 10, 15, 20, 30, 40, 50]
    inC = xr.apply_ufunc(np.digitize, inD, class_bins)

    # Plot the figure, remove grid and ticks
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])

    ### TODO: add the year to the map, may need to experiment with the location depend on geography
    ax.text(0, 5, year, fontsize=40, color="white")

    plt.margins(0, 0)
    if out_file != "":
        # plt.imsave(out_file, inC[0,:,:], cmap=plt.get_cmap('magma'))
        plt.imshow(inC[0, :, :], cmap=plt.get_cmap("magma"))
        fig.savefig(out_file, dpi=100, bbox_inches="tight", pad_inches=0)
    else:
        # https://matplotlib.org/stable/tutorials/colors/colormaps.html
        plt.imshow(inC[0, :, :], cmap=plt.get_cmap("magma"))


cur_file = all_files[0]
out_file = os.path.join(out_map_folder, os.path.basename(cur_file))

print(out_file)
map_viirs(cur_file, "")
# map_viirs(cur_file, out_file)

In [ ]:
for cur_file in all_files:
    out_file = os.path.join(out_map_folder, os.path.basename(cur_file))
    map_viirs(cur_file, out_file)

In [ ]:
kwargs = {"duration": 0.3}
images = []
all_tifs = [os.path.join(out_map_folder, x) for x in os.listdir(out_map_folder)]
all_tifs.sort()
for filename in all_tifs:
    images.append(imageio.imread(filename))

imageio.mimsave("%s_timelapse.gif" % out_map_folder, images, **kwargs)

In [ ]:
"%s_timelapse.gif" % out_map_folder